In [1]:
import numpy as np
import skimage
import matplotlib.pyplot as plt
from tempfile import TemporaryFile
from PIL import Image, ImageEnhance
from os import listdir
import imghdr
from skimage.transform import rotate, AffineTransform, warp
from skimage import filters
from torchvision import transforms
import cv2
import scipy
import random

In [2]:
rmdir '../Data_full/.ipynb_checkpoints'

rmdir: failed to remove './Data_full/.ipynb_checkpoints': No such file or directory


In [35]:
### Load Data and Handle Preprocessing

# Input: Folder with all the .png images
# Output: Images and respective labels
# Works?: Yes
def loadImages(path):
    imagesList = listdir(path)
    imgs = {}
    for image in imagesList:
      if imghdr.what(path + image) == 'png':
        if (image[6].isalpha()): # only add 5 of each image, only add alphabetical values
          img = Image.open(path + image)
          imgs[image[0:len(image) - 4]] = img
          # labels.append(ord(image[6]) - ord('a')) # assumes that filename structure is 'handx_[label]_....'
    return imgs

# Convert png img array to array np arrays
# Input: PNG image array
# Output: a list of numpy images
# Works?: Yes
def ImagesToArray(imgs):
  imgs_array = {}
  for imgName in imgs.keys():
    img_array = np.array(imgs[imgName])
    imgs_array[imgName] = img_array
  return imgs_array

# Zero pad all images
# Input: list of numpy images
# Output: numpy array of N 600x600 images with 3 channels
# Works?: Yes, but might not be necessary
def shape600(imgs):
  # reshaped_array = np.zeros((len(x), 600, 600, 3))
  reshaped_array = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    x_pad_width = (600 - img.shape[0])//2
    y_pad_width = (600 - img.shape[1])//2
    # reshaped_array[i,:,:,:] = np.pad(img, ((x_pad_width, x_pad_width + (img.shape[0])%2), (y_pad_width, y_pad_width+(img.shape[1]%2)), (0,0)), constant_values=img[0][0][0])
    new_img = (np.pad(img, ((x_pad_width, x_pad_width + (img.shape[0])%2), (y_pad_width, y_pad_width+(img.shape[1]%2)), (0,0)), constant_values=img[0][0][0])).astype('uint8')
    reshaped_array[imgName] = new_img
    img_save = Image.fromarray(new_img, 'RGB')
    img_save.save('./FinalImages/' + imgName + '.png')
  return reshaped_array

# Normalizes images... based on... what?
# Input: image array
# Output: a list of numpy arrays
# Works?: ??? Not entirely sure if this is the correct method though, based on online implementations of AlexNet
def Normalize(imgs):
  new_imgs = []
  for img in imgs:
      # flat_img = img.flatten()
      m = np.mean(img)
      std = np.std(img)
      img = (img-m)/std
      new_imgs.append(img)
  return new_imgs

def blur(imgs, sd=1):
  # filtered_img = np.zeros((imgs.shape[0], 600, 600, 3))
  filtered_img = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    new_img = skimage.filters.gaussian(img, sigma=sd)
    filtered_img[imgName + '_blurred'] = new_img
    img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    img_save.save('./FinalImages/' + imgName + '_blurred' + '.png')
  return filtered_img

def Scale(imgs):
  # scaled_images = np.zeros((len(imgs), 600, 600, 3))
  scaled_images = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    # ratio = random.randrange(.2, .5, .1)
    ratio = random.choice([0.1, 0.2, 0.3, 0.4])
    x = int(ratio * 600 / 2)
    scaled = img[x:600-x, x:600-x]
    # scaled = cv2.imread(scaled)
    new_img = cv2.resize(scaled, dsize=(600, 600), interpolation=cv2.INTER_CUBIC)
    scaled_images[imgName + '_scaled'] = new_img
    img_save = Image.fromarray(new_img, 'RGB')
    img_save.save('./FinalImages/' + imgName + '_scaled' + '.png')
  return scaled_images
    

def Rotate30(imgs): 
  # rot30_imgs = np.empty([imgs.shape[0], imgs.shape[1], imgs.shape[2], imgs.shape[3]])
  rot30_imgs = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    rand_dir = random.choice([-1, 1])
    new_img = rotate(img, rand_dir * 30) 
    rot30_imgs[imgName + '_rotated'] = new_img
    img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    img_save.save('./FinalImages/' + imgName + '_rotated' + '.png')
  return rot30_imgs

def VerticalFlip(imgs): 
  # flip_imgs = np.empty([imgs.shape[0], imgs.shape[1], imgs.shape[2], imgs.shape[3]])
  flip_imgs = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    new_img = np.fliplr(img)
    flip_imgs[imgName + '_flipped'] = new_img
    img_save = Image.fromarray(new_img, 'RGB')
    img_save.save('./FinalImages/' + imgName + '_fipped' + '.png')
  return flip_imgs

def Translation(imgs): 
  # trans_imgs = np.empty([imgs.shape[0], imgs.shape[1], imgs.shape[2], imgs.shape[3]])
  trans_imgs = {}
  for imgName in imgs.keys():
    img = imgs[imgName]
    rand_x = random.randrange(-150, 150, 50)
    rand_y = random.randrange(-150, 150, 50)
    transform = AffineTransform(translation=(rand_x,rand_y))
    new_img = warp(img,transform, mode="constant")  
    trans_imgs[imgName + '_flipped'] = new_img
    img_save = Image.fromarray((new_img * 255).astype('uint8'), 'RGB')
    img_save.save('./FinalImages/' + imgName + '_translated' + '.png')
  return trans_imgs
####################################################

# load png_images from Data_full foler
path = "../Data_full/"

# images in an array named imgs
imgs_png = loadImages(path)

# Step 1 convert png_images to np arrays
imgs_array_before = ImagesToArray(imgs_png)

# # Step 2 Normalize images
# # imgs_array_normalize = Normalize(imgs_array_before)

# # Step 3 Reshape the images
imgs_array_reshaped = shape600(imgs_array_before)

# # Step 4 Blur the images
blurred_imgs = blur(imgs_array_reshaped, 10)

# Step 5 Scale the images
scaled_imgs = Scale(imgs_array_reshaped)

# Step 6 Flip the images
flipped_imgs = VerticalFlip(imgs_array_reshaped)

# Step 7 Add translation to images
translated_imgs = Translation(imgs_array_reshaped)

# Step 8 Rotate 30 Degrees
rotated30_imgs = Rotate30(imgs_array_reshaped)

# fig, ax = plt.subplots(1, 7, figsize=(15,10))
# ax[0].imshow(imgs_array_before[0]) # Step 1
# ax[1].imshow(imgs_array_reshaped[0]) # Step 2
# ax[2].imshow(blurred_imgs[0]) # Step 3
# ax[3].imshow(scaled_imgs[0]) # Step 4
# ax[4].imshow(flipped_imgs[0]) # Step 5
# ax[5].imshow(translated_imgs[0]) # Step 6
# ax[6].imshow(rotated30_imgs[0]) # Step 7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
